In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[6],4
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,257,257


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000155                    1 -0.000263
0.000750        2 -0.000126                    2 -0.000354
0.001052        3 -0.000053                    3 -0.000153
0.001476        4  0.000048                    4 -0.000084
0.002070        5  0.000179                    5  0.000002
0.002904        6  0.000348                    6  0.000107
0.004074        7  0.000562                    7  0.000243
0.005714        8  0.000830                    8  0.000425
0.008015        9  0.001162                    9  0.000654
0.011243       10  0.001570                   10  0.000932
0.015771       11  0.002067                   11  0.001268
0.022122       12  0.002670                   12  0.001675
0.031031       13  0.003391                   13  0.002162
0.043528       14  0.004146                   14  0.002668
0.061057       15  0.004815                   15  0.003119
0.085645       16  0.005448                   16  0.003554
0.120136       17  0.006138                   17  0.004024
0.168516       18  0.006890                   18  0.004522
0.236378       19  0.007711                   19  0.005043
0.331549       20  0.008600                   20  0.005590
0.465100       21  0.009457                   21  0.006167
0.652400       22  0.008676                   22  0.005792
0.915100       23  0.006141                   23  0.004173
1.283650       24  0.003864                   24  0.002648
1.800600       25  0.002291                   25  0.001571
2.525700       26  0.001237                   26  0.000837
3.542800       27  0.000554                   27  0.000353
4.969550       28  0.000173                   28  0.000077
6.970850       29 -0.000012                   29 -0.000062
9.778100       30 -0.000120                   30 -0.000146
13.715850      31 -0.000195                   31 -0.000207
19.239350      32 -0.000246                   32 -0.000248
26.987250      33 -0.000265                   33 -0.000264
37.855300      34 -0.000261                   34 -0.000258
53.100050      35 -0.000253                   35 -0.000262
73.887500      36 -0.000247                   36 -0.000267
97.662500      37 -0.000247                   37 -0.000264
121.437500     38 -0.000259                   38 -0.000265
145.212500     39 -0.000279                   39 -0.000268
168.987500     40 -0.000298                   40 -0.000271
192.762500     41 -0.000314                   41 -0.000275
216.537500     42 -0.000329                   42 -0.000286
240.312500     43 -0.000343                   43 -0.000299
264.087500     44 -0.000356                   44 -0.000310
287.862500     45 -0.000365                   45 -0.000316
311.637500     46 -0.000342                   46 -0.000293
335.412500     47 -0.000260                   47 -0.000214
359.187500     48 -0.000120                   48 -0.000086
382.962500     49  0.000076                   49  0.000086
406.737500     50  0.000355                   50  0.000322
430.512500     51  0.000742                   51  0.000640
454.287500     52  0.001271                   52  0.001072
478.062500     53  0.001971                   53  0.001654
501.837500     54  0.002870                   54  0.002415
525.612500     55  0.003985                   55  0.003334
549.387500     56  0.005319                   56  0.004462
573.162500     57  0.006859                   57  0.005801
596.937500     58  0.008571                   58  0.007338
620.712500     59  0.010408                   59  0.009039
644.487500     60  0.012312                   60  0.010808
668.262500     61  0.014225                   61  0.012585
692.037500     62  0.016095                   62  0.014366
715.812500     63  0.017880                   63  0.016107
739.587500     64  0.019550                   64  0.017772
763.362500     65  0.021092                   65  0.019339
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -32.747664  0.000000e+00 -32.747664 -32.762498   
0.000624    2     -32.747664  3.533345e-08 -32.747664 -32.762498   
0.000876    3     -32.747664  5.092928e-08 -32.747664 -32.762498   
0.001229    4     -32.747664  7.719870e-08 -32.747664 -32.762498   
0.001723    5     -32.747664  1.214675e-07 -32.747664 -32.762498   
0.002417    6     -32.747664  1.958874e-07 -32.747664 -32.762498   
0.003391    7     -32.747664  3.207319e-07 -32.747664 -32.762498   
0.004757    8     -32.747664  5.296845e-07 -32.747664 -32.762498   
0.006672    9     -32.747665  8.785336e-07 -32.747664 -32.762498   
0.009359    10    -32.747665  1.459702e-06 -32.747663 -32.762499   
0.013128    11    -32.747665  2.425853e-06 -32.747663 -32.762499   
0.018415    12    -32.747665  4.028688e-06 -32.747661 -32.762499   
0.025830    13    -32.747666  6.682259e-06 -32.747659 -32.762500   
0.036232    14    -32.747666  1.106324e-05 -32.747655 -32.762500   
0.050823    15    -32.747666  1.814879e-05 -32.747648 -32.762501   
0.071291    16    -32.747665  2.922940e-05 -32.747636 -32.762501   
0.100000    17    -32.747664  4.625490e-05 -32.747617 -32.762502   
0.140271    18    -32.747660  7.235875e-05 -32.747588 -32.762501   
0.196760    19    -32.747654  1.122915e-04 -32.747542 -32.762500   
0.275997    20    -32.747643  1.732556e-04 -32.747470 -32.762496   
0.387100    21    -32.747622  2.660238e-04 -32.747356 -32.762488   
0.543100    22    -32.747587  4.056581e-04 -32.747182 -32.762474   
0.761700    23    -32.747544  5.866388e-04 -32.746957 -32.762457   
1.068500    24    -32.747512  7.782820e-04 -32.746734 -32.762453   
1.498800    25    -32.747503  9.665340e-04 -32.746537 -32.762468   
2.102400    26    -32.747522  1.149343e-03 -32.746373 -32.762508   
2.949000    27    -32.747574  1.325190e-03 -32.746249 -32.762577   
4.136600    28    -32.747665  1.493495e-03 -32.746171 -32.762680   
5.802500    29    -32.747800  1.662814e-03 -32.746137 -32.762824   
8.139200    30    -32.747992  1.851319e-03 -32.746140 -32.763020   
11.417000   31    -32.748261  2.074003e-03 -32.746187 -32.763288   
16.014700   32    -32.748636  2.342899e-03 -32.746293 -32.763654   
22.464000   33    -32.749158  2.677980e-03 -32.746481 -32.764152   
31.510500   34    -32.749907  3.142266e-03 -32.746764 -32.764847   
44.200100   35    -32.751014  3.857382e-03 -32.747156 -32.765835   
62.000000   36    -32.752665  4.975411e-03 -32.747690 -32.767344   
85.775000   37    -32.755010  6.624023e-03 -32.748386 -32.769562   
109.550000  38    -32.757474  8.393070e-03 -32.749081 -32.771902   
133.325000  39    -32.760018  1.020646e-02 -32.749812 -32.774294   
157.100000  40    -32.762634  1.203448e-02 -32.750599 -32.776681   
180.875000  41    -32.765315  1.387744e-02 -32.751437 -32.779069   
204.650000  42    -32.768061  1.573848e-02 -32.752322 -32.781475   
228.425000  43    -32.770863  1.761406e-02 -32.753249 -32.783956   
252.200000  44    -32.773719  1.950453e-02 -32.754214 -32.786532   
275.975000  45    -32.776630  2.141371e-02 -32.755217 -32.789186   
299.750000  46    -32.779609  2.336543e-02 -32.756244 -32.791916   
323.525000  47    -32.782722  2.551373e-02 -32.757208 -32.794775   
347.300000  48    -32.786053  2.811200e-02 -32.757941 -32.797786   
371.075000  49    -32.789617  3.133740e-02 -32.758279 -32.800984   
394.850000  50    -32.793496  3.543099e-02 -32.758065 -32.804460   
418.625000  51    -32.797775  4.070878e-02 -32.757066 -32.808269   
442.400000  52    -32.802495  4.752023e-02 -32.754975 -32.812415   
466.175000  53    -32.807689  5.629302e-02 -32.751396 -32.816954   
489.950000  54    -32.813365  6.751863e-02 -32.745846 -32.821914   
513.725000  55    -32.819498  8.173364e-02 -32.737764 -32.827288   
537.500000  56    -32.826027  9.948647e-02 -32.726540 -32.832950   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')